In [ ]:
# Use DeepSurv from the repo
import sys
sys.path.append('../deepsurv')
import deep_surv

from deepsurv_logger import DeepSurvLogger, TensorboardLogger
import utils
import viz

import numpy as np
import pandas as pd

import lasagne
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
for i in range(1,200):
    train_dataset_fp_trt_1 = './data_gen_partial_censor_40_trt_1_'+str(i)+'.csv'
    train_df_trt_1 = pd.read_csv(train_dataset_fp_trt_1)
    train_dataset_fp_trt_0 = './data_gen_partial_censor_40_trt_0_'+str(i)+'.csv'
    train_df_trt_0 = pd.read_csv(train_dataset_fp_trt_0)
    
    # event_col is the header in the df that represents the 'Event / Status' indicator
    # time_col is the header in the df that represents the event time
    def dataframe_to_deepsurv_ds(df, event_col = 'Event', time_col = 'Time'):
    # Extract the event and time columns as numpy arrays
    e = df[event_col].values.astype(np.int32)
    t = df[time_col].values.astype(np.float32)

    # Extract the patient's covariates as a numpy array
    x_df = df.drop([event_col, time_col], axis = 1)
    x = x_df.values.astype(np.float32)
    
    # Return the deep surv dataframe
    return {
        'x' : x,
        'e' : e,
        't' : t
    }

    # If the headers of the csv change, you can replace the values of 
    # 'event_col' and 'time_col' with the names of the new headers
    # You can also use this function on your training dataset, validation dataset, and testing dataset
    train_data_trt_1 = dataframe_to_deepsurv_ds(train_df_trt_1, event_col = 'Event', time_col= 'Time')
    train_data_trt_0 = dataframe_to_deepsurv_ds(train_df_trt_0, event_col = 'Event', time_col= 'Time')
    import theano
    hyperparams_trt_1 = {
    'L2_reg': 10.0,
    'batch_norm': True,
    'dropout': 0.4,
    'hidden_layers_sizes': [25, 25],
    'learning_rate': 1e-05,
    'lr_decay': 0.001,
    'momentum': 0.9,
    'n_in': train_data_trt_1['x'].shape[1],
    'standardize': False
    }
    
    hyperparams_trt_0 = {
    'L2_reg': 10.0,
    'batch_norm': True,
    'dropout': 0.4,
    'hidden_layers_sizes': [25, 25],
    'learning_rate': 1e-05,
    'lr_decay': 0.001,
    'momentum': 0.9,
    'n_in': train_data_trt_0['x'].shape[1],
    'standardize': False
    }
    
    # Create an instance of DeepSurv using the hyperparams defined above
    model_trt_1 = deep_surv.DeepSurv(**hyperparams_trt_1)
    model_trt_0 = deep_surv.DeepSurv(**hyperparams_trt_0)
    
    compute_hazards_trt_1 = theano.function(inputs = [model_trt_1.X],outputs = model_trt_1.partial_hazard)
    partial_hazards_trt_1_trt_1 = compute_hazards_trt_1(train_data_trt_1['x'])
    partial_hazards_trt_1_trt_0 = compute_hazards_trt_1(train_data_trt_0['x'])
    compute_hazards_trt_0 = theano.function(inputs = [model_trt_0.X],outputs = model_trt_0.partial_hazard)
    partial_hazards_trt_0_trt_1 = compute_hazards_trt_0(train_data_trt_1['x'])
    partial_hazards_trt_0_trt_0 = compute_hazards_trt_0(train_data_trt_0['x'])
    np.savetxt('./partial_hazards_partial_censor_40_trt_1_trt_1_'+str(i)+'.csv', partial_hazards_trt_1_trt_1, delimiter=",")
    np.savetxt('./partial_hazards_partial_censor_40_trt_1_trt_0_'+str(i)+'.csv', partial_hazards_trt_1_trt_0, delimiter=",")
    np.savetxt('./partial_hazards_partial_censor_40_trt_0_trt_0_'+str(i)+'.csv', partial_hazards_trt_0_trt_0, delimiter=",")
    np.savetxt('./partial_hazards_partial_censor_40_trt_0_trt_1_'+str(i)+'.csv', partial_hazards_trt_0_trt_1, delimiter=",")